# [Repurpose Covidlies]

Repurpose the COVIDLies dataset as a instruction dataset so to integrate it into the augmentation pipeline

# Setup

## Library import
We import all the required Python libraries

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os,sys
import joblib
from os import path

# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Common things
# from sklearn.metrics import classification_report
# from scipy import stats

# Visualizations
# import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_style("white") # darkgrid, whitegrid, dark, white, and ticks
# plt.figure(figsize=(7, 7))

# Autoreload extension
# if 'autoreload' not in get_ipython().extension_manager.loaded:
#     %load_ext autoreload
    
# %autoreload 2

In [3]:
# Examples seaborn
# with sns.axes_style("whitegrid"):
#     fig, axis = plt.subplots(1, 2, figsize=(20, 5), sharey=True)
#     fig.suptitle(f'Position distribution on splits')
#     sns.boxplot(ax=axis[0], data=df_prep, y='event', x='ith_pos', order=event_label_map.values())
#     sns.boxplot(ax=axis[1], data=pd.read_pickle(path.join(DATA_PATH, "stage2_test.pkl") ), y='event', x='ith_pos', order=event_label_map.values())

## Local library import
We import all the required local libraries libraries

In [4]:
# Include local library paths
import sys
# sys.path.append('path/to/local/lib') # uncomment and fill to import local libraries
# add project folders so local libraries can be imported
sys.path.insert(1, os.path.join(os.getcwd()  , '..'))
sys.path.insert(1, os.path.join(os.getcwd()  , '../src'))

# Import local libraries
# from plibs.utils import corrstats
# from src.plibs.utils import plots as myplots

In [5]:
# notebook misc functions
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

def displayAll(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):	
        display(df)  

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.


# Data import

Only 3256 tweets were recovered... verified also from original Joseph files (retrived from GDrive)

In [6]:
DATA_PATH = "../data/data/covidlies"

In [8]:
df = pd.read_csv(path.join(DATA_PATH, "covid_lies.tsv"), sep='\t')
print(df.shape)
df.head()

(6591, 9)


,misconception_id,misconception,tweet_id,label,tweet_text,tweet_favorite_count,user_id,user_name,user_followers_count
0,3,Coronavirus is genetically engineered.,1233965490948591616,na,NaN,NaN,NaN,NaN,NaN
1,30,Blowing conch shells destroys coronavirus pote...,1233907923765559296,na,Getting coronavirus and then coughing on peopl...,0.0,1.205737e+18,Cлава Сталин☭,58.0
2,57,Swans and dolphins swimming in Venice canals f...,1233911842910720000,na,@Barioth_a_bot そう…ですね…私は常時深海に居ります故…ふわふわ、という感触で...,0.0,3.694222e+08,ナバルデウス,1099.0
3,22,Cocaine cures coronavirus.,1233947734094290944,na,RT @TabascoHOY: #Coronavirus☣\n➡Los infectados...,0.0,1.628865e+08,Fx,489.0
4,32,Observing janata curfew will result in the red...,1233937085297332224,na,NaN,NaN,NaN,NaN,NaN


In [33]:
df[~df.tweet_text.isna()]

,misconception_id,misconception,tweet_id,label,tweet_text,tweet_favorite_count,user_id,user_name,user_followers_count
1,30,Blowing conch shells destroys coronavirus pote...,1233907923765559296,na,Getting coronavirus and then coughing on peopl...,0.0,1.205737e+18,Cлава Сталин☭,58.0
2,57,Swans and dolphins swimming in Venice canals f...,1233911842910720000,na,@Barioth_a_bot そう…ですね…私は常時深海に居ります故…ふわふわ、という感触で...,0.0,3.694222e+08,ナバルデウス,1099.0
3,22,Cocaine cures coronavirus.,1233947734094290944,na,RT @TabascoHOY: #Coronavirus☣\n➡Los infectados...,0.0,1.628865e+08,Fx,489.0
7,3,Coronavirus is genetically engineered.,1233952635725778944,na,Branco já nasce sem noção ou enfia ela no cu d...,10.0,2.134242e+08,viktoriia,595.0
9,50,It is safe for individuals infected with COVID...,1233920123183828992,na,"“Most people (not elderly, no underlying healt...",0.0,2.826966e+08,Eileen May,2913.0
...,...,...,...,...,...,...,...,...,...
6583,36,Chloroquine can cure coronavirus,1233916644877111296,na,Scientists say a cutting-edge but untested new...,18.0,1.667557e+07,The Sydney Morning Herald,845388.0
6586,50,It is safe for individuals infected with COVID...,1233949547715584000,na,One Spokane County resident is displaying symp...,9.0,3.934221e+06,The Spokesman-Review,59818.0
6587,19,Vitamin C cures coronavirus.,1233966049097285632,na,@ALBeforeAfter Orion is my favorite constellat...,5.0,2.224523e+08,K&A,67352.0
6589,61,Lions were freed to keep people off the street...,1233906211310825472,na,@LindaJones_NYC China is dirty.\nWhen people c...,0.0,8.885212e+17,Chatelaine,2508.0


In [34]:
df[~df.tweet_text.isna()].label.value_counts()

na     3056
pos     116
neg      84
Name: label, dtype: int64

In [35]:
# Using only entries with tweet text available
df = df[~df.tweet_text.isna()]

In [36]:
(df.misconception.unique())

array(['Blowing conch shells destroys coronavirus potency.',
       'Swans and dolphins swimming in Venice canals following COVID-19 lockdown.',
       'Cocaine cures coronavirus.',
       'Coronavirus is genetically engineered.',
       'It is safe for individuals infected with COVID-19 to go to work.',
       'Observing janata curfew will result in the reduction of covid-19 cases by 40%.',
       'Lions were freed to keep people off the streets in Moscow.',
       'The first person infected is a researcher named Huang Yanling.',
       'Actor Keith Lawrence Middlebrook has a cure for COVID-19.',
       'Viruses are wastes from cells that are poisoned by electromagnetic fields.',
       'Africans are more resistant to coronavirus.',
       'Coronavirus can only survive in cold temperatures.',
       'Vitamin C can prevent coronavirus circulation.',
       'Coronavirus is cured by shuanghuanglian.',
       'Cannabis protects against COVID-19.',
       'Methanol consumption cures or pro

In [37]:
print(df.label.describe())
print(df.label.unique())
df.label.value_counts()

count     3256
unique       3
top         na
freq      3056
Name: label, dtype: object
['na' 'neg' 'pos']


na     3056
pos     116
neg      84
Name: label, dtype: int64

In [31]:
df_val = pd.read_csv(path.join(DATA_PATH, "covid_lies.val.extended.tokenized.urldomain.tsv"), sep="\t")
print(df_val.shape)
display(df_val[df_val.tweet_text.isna()])
print(df_val.label.value_counts())
df_val.head()

(651, 7)


,tweet_id,tweet_text,label,tweet_processed,user_id,user_followers_count,tweet_favorite_count


na     612
pos     22
neg     17
Name: label, dtype: int64


,tweet_id,tweet_text,label,tweet_processed,user_id,user_followers_count,tweet_favorite_count
0,1233950193902608384,Can Lysol and Clorox products kill the novel c...,na,Can Lysol and Clorox products kill the novel c...,9.137786e+08,14827.0,0.0
1,1233970016690806784,"As fears over #coronavirus grow, CVS cautioned...",na,"As fears over coronavirus grow , CVS cautio...",2.458732e+08,1462.0,0.0
2,1233975282912022528,#covid-19 Another hope for drug. \n@ICMRDELHI ...,na,covid-19 Another hope for drug . \n @user ...,8.603675e+17,62.0,0.0
3,1233916644877111296,Scientists say a cutting-edge but untested new...,na,Scientists say a cutting - edge but untested n...,1.667557e+07,845388.0,18.0
4,1233941827566395392,@VGKnightinArmor @AnnasWorld3 @xqben @realDona...,na,@user @user @user @user Still has n't mutated ...,4.203783e+09,2043.0,0.0


In [32]:
df_train = pd.read_csv(path.join(DATA_PATH, "covid_lies.train.extended.tokenized.urldomain.tsv"), sep="\t")
print(df_train.shape)
display(df_train[df_train.tweet_text.isna()])
print(df_train.label.value_counts())
df_train.head()

(2605, 7)


,tweet_id,tweet_text,label,tweet_processed,user_id,user_followers_count,tweet_favorite_count


na     2444
pos      94
neg      67
Name: label, dtype: int64


,tweet_id,tweet_text,label,tweet_processed,user_id,user_followers_count,tweet_favorite_count
0,1233942503377686528,@FWhitfield @CNNSotu @BrookeBCNN @OutFrontCNN ...,na,@user @user @user @user @user @user @user What...,1.207575e+18,15.0,0.0
1,1233921165040726016,As a committee of The Federation for a Democra...,na,As a committee of The Federation for a Democra...,9.906354e+17,118.0,11.0
2,1233913926703812608,#Coronavirus #COVID19 Attached are the Global ...,na,Coronavirus COVID19 Attached are the Global Ca...,1.660128e+09,321.0,0.0
3,1233906989664632832,@TiffanyAlaniz @DrEricDing The only prevention...,na,@user @user The only prevention from any virus...,2.126882e+07,554.0,1.0
4,1233929870645825536,Can someone who has media attention tell peopl...,na,Can someone who has media attention tell peopl...,1.246761e+09,99.0,1.0


# Data processing
Put here the core of the notebook. Feel free to further split this section into subsections.

In [ ]:
import regex as re
import langdetect

In [41]:
dftmp = df.sample(5)
displayAll(dftmp)

,misconception_id,misconception,tweet_id,label,tweet_text,tweet_favorite_count,user_id,user_name,user_followers_count
2746,49,We're very close to a vaccine.,1233950426233479168,na,"If the coronavirus puts the global economy on lockdown, we're about to see a dangerous experiment.\n\nI mean,\nfirst we geo-engineer the climate by burning fossil fuels.\n\nNow we may be about to kick that global warming into overdrive.",1.0,4.111482e+08,CelloMom on Cars,751.0
5620,29,Clapping will kill coronavirus.,1233928410633768960,na,@barstoolsports Don’t stop him. Film it. Great idea. That’s why Coronavirus will spread. Because of morons that know better and do nothing.,8.0,1.072686e+18,CM Barrett,10.0
5899,11,Viruses are wastes from cells that are poisoned by electromagnetic fields.,1233926224969232384,na,"@tatazz_z Even in Japan, where vaccine is widely available, a lot of old people die every year because of influenza. It is important to note it is a lot more probable that old people are killed by influenza than corona virus.",0.0,4.497803e+08,Atsushi_Kato,115.0
4669,27,Drinking cow urine and applying cow dung on the body can cure covid-19.,1233981202459189248,na,"@adamhamdy Have we forgotten the spread in animals? Presumably livestock, wild animals and pets can pass COVID-19 to us. What about eating meat from infected animals. There are already reported cases of Coronavirus in dogs in China.",1.0,2.158274e+08,SimonVO.com,96.0
4891,9,Africans are more resistant to coronavirus.,1233917765804687360,na,"@stnr4u2 @charliekirk11 Actually, no, that is not true. The Chinese did a quick study of 45k of those who have been diagnosed with this coronavirus and the most likely to die are 35 and up. \n\nKids, oddly enough, are less likely to get it because of their exposure to other corona viruses at school. 1/",1.0,8.337677e+17,"GCH, The guy who makes music",388.0


In [66]:
# To Extract mentions
# mention_pattern = re.compile(r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9_]+)")
pattern_mention = re.compile(r"(?<=@)\w+")
pattern_hashtag = re.compile(r"#(\w+)")
def extract_mentions(txt):
    return pattern_mention.findall(txt)  
def extract_hashtags(txt):
    return pattern_hashtag.findall(txt)   

In [67]:
def preprocess(df):
    df["mentions"] = df.tweet_text.map(extract_mentions)
    df["hashtags"] = df.tweet_text.map(extract_hashtags)
    # df["lang"] = df.tweet_text.map(langdetect.detect)
    return df

In [68]:
df = preprocess(df)
df

/tmp/ipykernel_25324/2686072943.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["mentions"] = df.tweet_text.map(extract_mentions)
/tmp/ipykernel_25324/2686072943.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hashtags"] = df.tweet_text.map(extract_hashtags)


,misconception_id,misconception,tweet_id,label,tweet_text,tweet_favorite_count,user_id,user_name,user_followers_count,mentions,lang,hashtags
1,30,Blowing conch shells destroys coronavirus pote...,1233907923765559296,na,Getting coronavirus and then coughing on peopl...,0.0,1.205737e+18,Cлава Сталин☭,58.0,[],en,[]
9,50,It is safe for individuals infected with COVID...,1233920123183828992,na,"“Most people (not elderly, no underlying healt...",0.0,2.826966e+08,Eileen May,2913.0,[],en,[]
13,32,Observing janata curfew will result in the red...,1233921523695652864,na,15-20% of people with COVID-19 may require hos...,0.0,1.520276e+08,DebbieT 🐝 🇺🇸,182.0,[],en,[]
15,61,Lions were freed to keep people off the street...,1233919886839222272,na,A sheriff's office tried to get meth off the s...,0.0,2.018010e+07,WBAL NewsRadio 1090 and FM 101.5,71355.0,[],en,[]
16,5,The first person infected is a researcher name...,1233993586338926592,na,"In China’s War on the Coronavirus, a Community...",0.0,8.521893e+08,Cool Breeze,153.0,[],en,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
6583,36,Chloroquine can cure coronavirus,1233916644877111296,na,Scientists say a cutting-edge but untested new...,18.0,1.667557e+07,The Sydney Morning Herald,845388.0,[],en,[]
6586,50,It is safe for individuals infected with COVID...,1233949547715584000,na,One Spokane County resident is displaying symp...,9.0,3.934221e+06,The Spokesman-Review,59818.0,[],en,[]
6587,19,Vitamin C cures coronavirus.,1233966049097285632,na,@ALBeforeAfter Orion is my favorite constellat...,5.0,2.224523e+08,K&A,67352.0,[ALBeforeAfter],en,[]
6589,61,Lions were freed to keep people off the street...,1233906211310825472,na,@LindaJones_NYC China is dirty.\nWhen people c...,0.0,8.885212e+17,Chatelaine,2508.0,[LindaJones_NYC],en,[]


In [54]:
df.lang.value_counts()

en    3034
ja      64
es      43
th      24
pt      19
id      14
ar      11
so       8
de       7
ko       6
fi       5
fr       4
it       4
da       2
lv       2
ca       2
hi       1
tl       1
hr       1
ur       1
sw       1
et       1
ro       1
Name: lang, dtype: int64

In [56]:
df = df[df.lang =='en']
print(df.shape)
print(df.label.value_counts())
df


(3034, 11)
na     2842
pos     110
neg      82
Name: label, dtype: int64


,misconception_id,misconception,tweet_id,label,tweet_text,tweet_favorite_count,user_id,user_name,user_followers_count,mentions,lang
1,30,Blowing conch shells destroys coronavirus pote...,1233907923765559296,na,Getting coronavirus and then coughing on peopl...,0.0,1.205737e+18,Cлава Сталин☭,58.0,[],en
9,50,It is safe for individuals infected with COVID...,1233920123183828992,na,"“Most people (not elderly, no underlying healt...",0.0,2.826966e+08,Eileen May,2913.0,[],en
13,32,Observing janata curfew will result in the red...,1233921523695652864,na,15-20% of people with COVID-19 may require hos...,0.0,1.520276e+08,DebbieT 🐝 🇺🇸,182.0,[],en
15,61,Lions were freed to keep people off the street...,1233919886839222272,na,A sheriff's office tried to get meth off the s...,0.0,2.018010e+07,WBAL NewsRadio 1090 and FM 101.5,71355.0,[],en
16,5,The first person infected is a researcher name...,1233993586338926592,na,"In China’s War on the Coronavirus, a Community...",0.0,8.521893e+08,Cool Breeze,153.0,[],en
...,...,...,...,...,...,...,...,...,...,...,...
6583,36,Chloroquine can cure coronavirus,1233916644877111296,na,Scientists say a cutting-edge but untested new...,18.0,1.667557e+07,The Sydney Morning Herald,845388.0,[],en
6586,50,It is safe for individuals infected with COVID...,1233949547715584000,na,One Spokane County resident is displaying symp...,9.0,3.934221e+06,The Spokesman-Review,59818.0,[],en
6587,19,Vitamin C cures coronavirus.,1233966049097285632,na,@ALBeforeAfter Orion is my favorite constellat...,5.0,2.224523e+08,K&A,67352.0,[ALBeforeAfter],en
6589,61,Lions were freed to keep people off the street...,1233906211310825472,na,@LindaJones_NYC China is dirty.\nWhen people c...,0.0,8.885212e+17,Chatelaine,2508.0,[LindaJones_NYC],en


In [92]:
stance_mapping = {'pos': "in favor of", 'neg': "against", 'na': "neutral in regard of"}

def instructify(e:dict ):
    _include = f"mentions: [{', '.join(['@'+m for m in e.mentions])}]" if e.mentions else ""
    _include = _include + ((" and " if _include else "") +  (f"hashtags: [{', '.join(['#'+m for m in e.hashtags])}]") if e.hashtags else "")
    if _include:
        _include = "Include the following " + _include

    rs = {
        "instruction": f"Generate a tweet related to COVID-19 that have a stance {stance_mapping[e.label]} '{e.misconception}' {_include}",
        "context": f"",
        "category": 'stance_generation',
        "response": e["tweet_text"],
        "model": 'human',
    }
    
    return rs

In [94]:
for index, e in df.sample(3).iterrows():
    print(e.mentions)
    print(e.hashtags)
    displayAll(instructify(e))

[]
[]


{'instruction': "Generate a tweet related to COVID-19 that have a stance neutral in regard of 'Vitamin C can prevent coronavirus circulation.' ",
 'context': '',
 'category': 'stance_generation',
 'response': 'PSA: to prevent spread of coronavirus eat ass',
 'model': 'human'}

[]
['coronavirus']


{'instruction': "Generate a tweet related to COVID-19 that have a stance neutral in regard of 'A Malabar civet was spotted walking the streets of Meppayur, India following COVID-19 lockdown' Include the following hashtags: [#coronavirus]",
 'context': '',
 'category': 'stance_generation',
 'response': 'This is impossible that there are only 3 cases related to COVID-19 in India. The number is low due to:\n1. Gross underreporting by Gov\n2. Over the Counter treatment by infected people (likely)\nThis is going to blow up sooner than expected \n#coronavirus',
 'model': 'human'}

['JamesSt29455524', 'Nerdrotics', 'SCReviewsmang', 'RandomMnky', 'IvanFilmFix', 'JessiMilestone', 'DDayCobra', 'JosiahRises', 'OdinsMovieBlog', 'startrekcbs', 'FandomCollectiv', 'EnterHacker', 'Ruth64933323', 'doomcock', 'Midnights_Edge', 'nation_orville', 'JessieHair', 'FandomMenaceGrl', 'FandomMenaceYT', 'FandomMenace', 'Join_TFM', 'PopCulMinefield', 'OutSpok61328120', 'jeremy_pooper', 'YTPOOC', 'MagicPoopCannon', 'theworldofcrap', 'WhatCrappens', 'priceoreason', 'netflix', 'StarTrekNetflix', 'StarTrek', 'Picard', 'starwars', 'LuciferNetflix']
[]


{'instruction': "Generate a tweet related to COVID-19 that have a stance neutral in regard of 'Salt water protects from coronavirus.' Include the following mentions: [@JamesSt29455524, @Nerdrotics, @SCReviewsmang, @RandomMnky, @IvanFilmFix, @JessiMilestone, @DDayCobra, @JosiahRises, @OdinsMovieBlog, @startrekcbs, @FandomCollectiv, @EnterHacker, @Ruth64933323, @doomcock, @Midnights_Edge, @nation_orville, @JessieHair, @FandomMenaceGrl, @FandomMenaceYT, @FandomMenace, @Join_TFM, @PopCulMinefield, @OutSpok61328120, @jeremy_pooper, @YTPOOC, @MagicPoopCannon, @theworldofcrap, @WhatCrappens, @priceoreason, @netflix, @StarTrekNetflix, @StarTrek, @Picard, @starwars, @LuciferNetflix]",
 'context': '',
 'category': 'stance_generation',
 'response': "@JamesSt29455524 @Nerdrotics @SCReviewsmang @RandomMnky @IvanFilmFix @JessiMilestone @DDayCobra @JosiahRises @OdinsMovieBlog @startrekcbs @FandomCollectiv @EnterHacker @Ruth64933323 @doomcock @Midnights_Edge @nation_orville @JessieHair @FandomMenaceGr

In [96]:
rs = [instructify(e) for index, e in df.query("label =='pos'").iterrows()] + \
[instructify(e) for index, e in df.query("label =='neg'").iterrows()] + \
[instructify(e) for index, e in df.query("label =='na'").sample(90).iterrows()]

rs


[{'instruction': "Generate a tweet related to COVID-19 that have a stance in favor of 'Dean Koontz predicted the pandemic in his 1981 novel The Eyes of Darkness.' ",
  'context': '',
  'category': 'stance_generation',
  'response': 'People think this 1981 horror novel predicted the coronavirus outbreak https://t.co/NOb0oPBEPA',
  'model': 'human'},
 {'instruction': "Generate a tweet related to COVID-19 that have a stance in favor of 'We're very close to a vaccine.' Include the following mentions: [@SpeakerPelosi]",
  'context': '',
  'category': 'stance_generation',
  'response': '@SpeakerPelosi I understand Israel will have a vaccine for coronavirus in a few weeks.',
  'model': 'human'},
 {'instruction': "Generate a tweet related to COVID-19 that have a stance in favor of 'Coronavirus is genetically engineered.' ",
  'context': '',
  'category': 'stance_generation',
  'response': 'The corona virus is a government made virus that is being used for population control.',
  'model': 'huma

In [110]:
rs_ds = pd.DataFrame.from_dict(rs)
print(rs_ds.shape)
rs_ds

(282, 5)


,instruction,context,category,response,model
0,Generate a tweet related to COVID-19 that have...,,stance_generation,People think this 1981 horror novel predicted ...,human
1,Generate a tweet related to COVID-19 that have...,,stance_generation,@SpeakerPelosi I understand Israel will have a...,human
2,Generate a tweet related to COVID-19 that have...,,stance_generation,The corona virus is a government made virus th...,human
3,Generate a tweet related to COVID-19 that have...,,stance_generation,"@MusicThaman @alluarjun Presently, the effects...",human
4,Generate a tweet related to COVID-19 that have...,,stance_generation,@hughriminton @hamishNews\n@RonniSalt @DoctorK...,human
...,...,...,...,...,...
277,Generate a tweet related to COVID-19 that have...,,stance_generation,Coronavirus: the new disease Covid-19 explaine...,human
278,Generate a tweet related to COVID-19 that have...,,stance_generation,Its kinda bad when the surgeon General cant ev...,human
279,Generate a tweet related to COVID-19 that have...,,stance_generation,People don’t realize that it’s not just the el...,human
280,Generate a tweet related to COVID-19 that have...,,stance_generation,@Reuters I think WHO should change its criteri...,human


In [111]:
displayAll(rs_ds.sample(5))

,instruction,context,category,response,model
112,"Generate a tweet related to COVID-19 that have a stance against 'Hand sanitiser sold commercially does not destroy the coronavirus.' Include the following mentions: [@StanleyNavas, @AishaYesufu]",,stance_generation,@StanleyNavas @AishaYesufu This is untrue. Hand sanitizer will absolutely kill the covid-19 virus.,human
15,Generate a tweet related to COVID-19 that have a stance in favor of 'Coronavirus is genetically engineered.',,stance_generation,"What are the odds that a SARS-like coronavirus with overlapping genetics from HIV mutated and crossed over into humans, next door to a laboratory which had been enhancing coronavirus with HIV for over a decade? And conversely, what are the odds it leaked out of the laboratory?",human
249,"Generate a tweet related to COVID-19 that have a stance neutral in regard of 'Jamaat cannot become a source of infection.' Include the following mentions: [@TraderSpekt, @PDChina]",,stance_generation,"@TraderSpekt @PDChina They are still receiving treatment in the hospital. There is a process of infection and outbreak of the disease. The patients are not infected centrally, but gradually. The treatment must be in order.",human
130,Generate a tweet related to COVID-19 that have a stance against 'COVID-19 is only as deadly as the seasonal flu.',,stance_generation,"Many in the media have told us not to worry because 60,000 people a year die of the flu.\n\nEach year the flu infects 2 to 11% of the population.\n\nExperts think that 40 to 70 percent of people around the world will be infected with the virus that causes COVID-19 this year. \n4/15",human
201,Generate a tweet related to COVID-19 that have a stance neutral in regard of 'We're very close to a vaccine.',,stance_generation,"I don’t need a vaccine for coronavirus, I have AirPods",human


In [113]:
rs_ds.to_csv(path.join("../data/data/detection_dataset/", "repurposeds_stance_generation.tsv"), sep="\t")